In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import statsmodels.api as sm
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, plot_importance
from tqdm.notebook import tqdm
import scienceplots

import os
import sys
from dotenv import load_dotenv

load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")

# Import main utility functions
sys.path.insert(0, rf'{REPO_PATH}src')
from utils.main_utils import load_processed
from utils.forecast_utils import mean_directional_accuracy
plt.style.use('science')

### Import data

In [ ]:
FUTURES = ['CLc1', 'LCOc1']
TOPICS = ['CRU', 'CWP', 'CEN']

dfs = load_processed(FUTURES)

df = dfs['CLc1']

display(df.head())

### Multiple linear regression

In [ ]:
# multiple linear regression
# X = df[df.filter(like='_SI').columns]
X = df.drop(columns=['TARGET_1'])

y = df['TARGET_1']

X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()

print(model.summary())



### LASSO and Ridge regression

In [ ]:

# scale variables
scaler = RobustScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

lamdas = np.logspace(-5, 0, 500)

coefs_lasso = []
CV_MSE_lasso = []

for i in tqdm(lamdas, desc='Fitting Lasso '):
    lasso = Lasso(alpha=i, max_iter=10000)
    lasso.fit(X_train, y_train)
    coefs_lasso.append(lasso.coef_)
    CV_MSE_lasso.append(mean_squared_error(y_test, lasso.predict(X_test)))


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5), dpi=200)


plot_y = [coefs_lasso, CV_MSE_lasso]
label_y = ['Mean Square Error', 'Coefficients']
vlines = [CV_MSE_lasso]

# get twilight colors
colormap = cm.get_cmap('twilight_r', 20)

# Set the color cycle to the twilight colormap
axs[0].set_prop_cycle(color=colormap(np.linspace(0, 1, 8)))
axs[1].set_prop_cycle(color=colormap(np.linspace(0, 1, 2)))

for i, ax in enumerate(axs.flatten()):
    ax.plot(lamdas, plot_y[i], color='black' if i == 1 else None, lw=1.2)
    ax.set_xscale('log')
    # adjust tick size
    ax.tick_params(axis='both', which='major', labelsize=13)
    ax.set_xlabel('$\lambda$', fontsize=15)
    ax.set_ylabel(label_y[1] if i % 2 == 0 else label_y[0], fontsize=15)
    ax.axvline(
        lamdas[np.argmin(vlines[0] if i < 2 else vlines[1])], 
        color='red', 
        linestyle='-.',
        lw=1
    )

# set color on axs[1] line

fig.tight_layout()
fig.savefig('LASSO_Results.png')

# get coefficients for the best lambda
lasso = Lasso(alpha=lamdas[np.argmin(CV_MSE_lasso)], max_iter=10000)
lasso.fit(X_train, y_train)
print('Intercept for the best lambda:', lasso.intercept_)
print('MSE for the best lambda:', mean_squared_error(y_test, lasso.predict(X_test)))


In [ ]:
lasso_coefs = {col: 1 if coef != 0.0 else 0 for col, coef in zip(X.columns, lasso.coef_)}

# print the number of non-zero coefficients
print(f'Number of non-zero coefficients: {sum(lasso_coefs.values())}')
print(f'Number of zero coefficients: {len(lasso_coefs) - sum(lasso_coefs.values())}')

# remove all RV_LAG features
lasso_coefs = {k: v for k, v in lasso_coefs.items() if 'RV_LAG' not in k}

with open('feature_filters/lasso_coefs.json', 'w') as f:
    json.dump(lasso_coefs, f, indent=4)

In [ ]:
# OLS
X_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_const).fit()
X_const = sm.add_constant(X_test)

# LASSO
lasso = Lasso(alpha=lamdas[np.argmin(CV_MSE_lasso)], max_iter=10000)
lasso.fit(X_train, y_train)

# predict with XGBoost
xgb = XGBRegressor()
xgb.fit(X_train, y_train)


# Reduce number of features
features = X.columns[lasso.coef_ != 0]
X_train_BE = X_train[features]
X_test_BE = X_test[features]

# OLS Reduced
X_const_R = sm.add_constant(X_train_BE)
model_R = sm.OLS(y_train, X_const_R).fit()
X_const_R = sm.add_constant(X_test_BE)


# LASSO Reduced
lasso_R = Lasso(alpha=lamdas[np.argmin(CV_MSE_lasso)], max_iter=10000)
lasso_R.fit(X_train_BE, y_train)

# predict with Reduced XGBoost
xgb_R = XGBRegressor()
xgb_R.fit(X_train_BE, y_train)


predict_df = pd.DataFrame(
    {
        'True': y_test,
        'OLS': model.predict(X_const),
        'OLS_R': model_R.predict(X_const_R),
        'Lasso': lasso.predict(X_test),
        'Lasso_R': lasso_R.predict(X_test_BE),
        'XGBoost': xgb.predict(X_test),
        'XGBoost_R': xgb_R.predict(X_test_BE)
    }, index=y_test.index
)

window = 100
fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=200)
predict_df[['True', 'XGBoost', 'XGBoost_R']].iloc[-window:].plot(ax=ax, alpha=0.7)
ax.legend(frameon=False)

# sgb feature importance

fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=200)
plot_importance(xgb, ax=ax, max_num_features=20)
ax.legend(frameon=False)
ax.grid(alpha=0.2)


### Metrics

In [ ]:
metrics = pd.DataFrame(
    {
        'MSE': predict_df.apply(lambda x: mean_squared_error(predict_df['True'], x), axis=0),
        'DA': predict_df.apply(lambda x: mean_directional_accuracy(predict_df['True'], x), axis=0)
    }
)

display(metrics)